In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import itertools

### Little bit of data preprocessing. Rest happens inside cross validation function

In [2]:
data = pd.read_csv('../data/SMSSpamCollection', sep='\t', header=None)
data['Text'] = data[1].str.replace('[^\w\s]','')
data.columns = ['label', 'Full Text', 'Text']
data['Lower Case Text'] = data['Text'].str.lower()

In [3]:
labels, counts = np.unique(data['label'], return_counts=True)
encoder = preprocessing.LabelEncoder()
encoder.fit(labels[np.argsort(-counts)])
data['y'] = encoder.transform(data['label'])

In [4]:
np.random.seed(42)
mask_train = np.random.random(data.shape[0]) < 0.8
data_train = data[mask_train]
data_test = data.iloc[~mask_train, :]

### Load word2vec stuff

In [5]:
#up sample data train for word2vec vocabulary
countToIncrease_word = data_train[data_train['y'] == 0].shape[0] - data_train[data_train['y'] == 1].shape[0]
spamupsampled_word = data_train[data_train['y'] == 1].sample(n=countToIncrease_word, replace=True)
data_train_upsample_word2vec = pd.concat([spamupsampled_word, data_train])

In [6]:
count_vect_sing_word = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)
count_vect_sing_word.fit(data_train_upsample_word2vec['Lower Case Text'])
tokenizer_word = count_vect_sing_word.build_tokenizer()

In [7]:
VOCAB_SIZE = len(count_vect_sing_word.vocabulary_)
EMBEDDING_SIZE = 300
word_to_ix = count_vect_sing_word.vocabulary_

In [8]:
class TwoGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(TwoGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [9]:
CONTEXT_SIZE = 1
EMBEDDING_DIM = EMBEDDING_SIZE

In [10]:
word2vec_model = TwoGramLanguageModeler(VOCAB_SIZE, EMBEDDING_DIM, CONTEXT_SIZE)

In [11]:
MODEL_PATH = '../data/word_2vec_model'
word_embeddings = word2vec_model.embeddings
word2vec_model.load_state_dict(torch.load(MODEL_PATH))
word2vec_model.eval()

TwoGramLanguageModeler(
  (embeddings): Embedding(8111, 300)
  (linear1): Linear(in_features=300, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=8111, bias=True)
)

In [12]:
# count_vect_sing_word is a CountVectorizer
def _indicesForSentence(input_str, tokenizer = tokenizer_word, count_vect = count_vect_sing_word):
    input_str = list(filter(lambda x: x in count_vect.vocabulary_, tokenizer(input_str)))
    return torch.tensor([[word_to_ix[word]] for word in input_str], dtype=torch.long)

def getEmbedding(word_index_tensor, embedder):
    embedder(word_index_tensor)
def sentenceToNumpyInstance(input_str, embedder):
    embeddings = embedder(_indicesForSentence(input_str))
    if embeddings.shape == torch.Size([0]):
        return np.zeros(EMBEDDING_SIZE)
    else:
        x = torch.sum(embeddings, dim = 0)
        return torch.Tensor.numpy(x.detach())[0]
    
def word2vec_transform(data, embeddings, field = 'Lower Case Text'):
    return np.array(data[field].apply(sentenceToNumpyInstance, embedder=embeddings).values.tolist())

### Ending of Word2Vec initialization

### Cross-Validation

In [13]:
# dataset is supposed to be the normal trained data
## vectorizer is supposed to be the constructor
def crossval(dataset, model_func, parametersModel, embeddings = word_embeddings, vectorizer = None, 
             label_name = 'y', text_field = 'Lower Case Text', k_folds = 5): 
    aucs = {}
    folds = list(KFold(n_splits=k_folds).split(dataset))
    for fold in range(k_folds):
        cv_train = dataset.iloc[folds[fold][0]]
        cv_validate = dataset.iloc[folds[fold][1]]
    
        countToIncrease = cv_train[cv_train['y'] == 0].shape[0] - cv_train[cv_train['y'] == 1].shape[0]
        spamupsampled = cv_train[cv_train['y'] == 1].sample(n=countToIncrease, replace=True)
        cv_train = pd.concat([spamupsampled, cv_train])
        
        if vectorizer is not None:
            cv_train_features = vectorizer.fit_transform(cv_train[text_field])
            cv_train_y = cv_train[label_name]
            cv_validate_features = vectorizer.transform(cv_validate[text_field])
            cv_validate_y = cv_validate[label_name]
        else:
            # LOAD word to vec
            cv_train_features = word2vec_transform(cv_train, embeddings)
            cv_train_y = cv_train[label_name]
            cv_validate_features = word2vec_transform(cv_validate, embeddings)
            cv_validate_y = cv_validate[label_name]
        
        keys, values = zip(*parametersModel.items())
                
        for v in itertools.product(*values):    
            experiment = dict(zip(keys, v))
            m = model_func(**experiment)

            m.fit(cv_train_features, cv_train_y)
            y_preds = m.predict_proba(cv_validate_features)[:, 1]
            fpr, tpr, threshold = metrics.roc_curve(cv_validate_y, y_preds)
            auc_val = metrics.auc(fpr, tpr)  
            
            inside_key = ""

            for e in experiment:
                inside_key = inside_key + "_" + e + ':' + str(experiment[e])
            
            if not inside_key in aucs:
                aucs[inside_key] = []
                aucs[inside_key].append(auc_val)
            else:
                aucs[inside_key].append(auc_val)
    return aucs

In [14]:
LR_parameters = {'penalty': ['l1', 'l2'], 'C': [1, 5, 10]}

count_vect = CountVectorizer(min_df=2, stop_words=ENGLISH_STOP_WORDS)
tfidf_vect = TfidfVectorizer(min_df=2, stop_words=ENGLISH_STOP_WORDS)
bigram_count_vect = CountVectorizer(min_df=2, stop_words=ENGLISH_STOP_WORDS, ngram_range = (2,2))
bigram_tfidf_vect = TfidfVectorizer(min_df=2, stop_words=ENGLISH_STOP_WORDS, ngram_range = (2,2))
ngram_count_vect = CountVectorizer(min_df=2, stop_words=ENGLISH_STOP_WORDS, ngram_range = (1,2))
ngram_tfidf_vect = TfidfVectorizer(min_df=2, stop_words=ENGLISH_STOP_WORDS, ngram_range = (1,2))

In [15]:
aucs_count_default = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = count_vect)

In [16]:
aucs_tfidf_default = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = tfidf_vect)

In [17]:
aucs_count_bigram = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = bigram_count_vect)

In [18]:
aucs_tfidf_bigram = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = bigram_tfidf_vect)

In [19]:
aucs_count_ngram = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = ngram_count_vect)

In [20]:
aucs_tfidf_ngram = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = ngram_tfidf_vect)

In [21]:
# defaults to word 2 vec
aucs_word_2_vec = crossval(data_train, LogisticRegression, LR_parameters, vectorizer = None)

In [22]:
def processAUCsResults(aucs_dict):
    aucs = {}
    max_auc = -1
    confs = ""
    for e in aucs_dict:
        mean_auc = np.mean(aucs_dict[e])
        aucs['LR'+ e] = mean_auc
        if mean_auc >= max_auc:
            confs = e
            max_auc = mean_auc
    return aucs, confs

In [23]:
print(processAUCsResults(aucs_count_default))

({'LR_penalty:l1_C:1': 0.9785991319005006, 'LR_penalty:l1_C:5': 0.9808460754822054, 'LR_penalty:l1_C:10': 0.9819221616286926, 'LR_penalty:l2_C:1': 0.9889649363700389, 'LR_penalty:l2_C:5': 0.989242491721642, 'LR_penalty:l2_C:10': 0.9892710516402273}, '_penalty:l2_C:10')


In [24]:
print(processAUCsResults(aucs_tfidf_default))

({'LR_penalty:l1_C:1': 0.9768598339157105, 'LR_penalty:l1_C:5': 0.9791266912843237, 'LR_penalty:l1_C:10': 0.9795823131957532, 'LR_penalty:l2_C:1': 0.9886230964977036, 'LR_penalty:l2_C:5': 0.9896713875234973, 'LR_penalty:l2_C:10': 0.9898372840630074}, '_penalty:l2_C:10')


In [25]:
print(processAUCsResults(aucs_count_bigram))

({'LR_penalty:l1_C:1': 0.8805183412669118, 'LR_penalty:l1_C:5': 0.8888743386586515, 'LR_penalty:l1_C:10': 0.892444811244621, 'LR_penalty:l2_C:1': 0.9454633349453758, 'LR_penalty:l2_C:5': 0.9454256474566112, 'LR_penalty:l2_C:10': 0.9456754610212901}, '_penalty:l2_C:10')


In [26]:
print(processAUCsResults(aucs_tfidf_bigram))

({'LR_penalty:l1_C:1': 0.8791368988367461, 'LR_penalty:l1_C:5': 0.8891661832287282, 'LR_penalty:l1_C:10': 0.8910393322909524, 'LR_penalty:l2_C:1': 0.9445047820735075, 'LR_penalty:l2_C:5': 0.9442792840158292, 'LR_penalty:l2_C:10': 0.945394088013553}, '_penalty:l2_C:10')


In [27]:
print(processAUCsResults(aucs_count_ngram))

({'LR_penalty:l1_C:1': 0.9778788794787188, 'LR_penalty:l1_C:5': 0.9806106828778901, 'LR_penalty:l1_C:10': 0.9825216892117155, 'LR_penalty:l2_C:1': 0.989369208823606, 'LR_penalty:l2_C:5': 0.9899661401496254, 'LR_penalty:l2_C:10': 0.9901976985866373}, '_penalty:l2_C:10')


In [28]:
print(processAUCsResults(aucs_tfidf_ngram))

({'LR_penalty:l1_C:1': 0.9740342938614649, 'LR_penalty:l1_C:5': 0.977640473783844, 'LR_penalty:l1_C:10': 0.9778844338458083, 'LR_penalty:l2_C:1': 0.9858649183438238, 'LR_penalty:l2_C:5': 0.98763164865167, 'LR_penalty:l2_C:10': 0.9879518962811862}, '_penalty:l2_C:10')


In [29]:
print(processAUCsResults(aucs_word_2_vec))

({'LR_penalty:l1_C:1': 0.9261628207083324, 'LR_penalty:l1_C:5': 0.9223089021148987, 'LR_penalty:l1_C:10': 0.9212310045362887, 'LR_penalty:l2_C:1': 0.9261725350181171, 'LR_penalty:l2_C:5': 0.9238190049716044, 'LR_penalty:l2_C:10': 0.9228287948871827}, '_penalty:l2_C:1')


In [30]:
### run through with all training data on best parameters:

# need to upsample the data_train
# fit the vectorizer
# model should be an instantiated model
# returns the TEST AUC on normal rate and upsampled test
def train_test_run(data_train, data_test, model, embeddings = word_embeddings, vectorizer = None, 
                   label_name = 'y', text_field = 'Lower Case Text'):

    countToIncrease = data_train[data_train['y'] == 0].shape[0] - data_train[data_train['y'] == 1].shape[0]
    spamupsampled = data_train[data_train['y'] == 1].sample(n=countToIncrease, replace=True)
    data_train = pd.concat([spamupsampled, data_train])
    
    downsampled_ham = data_test[data_test['y'] == 0].sample(n=data_test[data_test['y'] == 1].shape[0])
    # the name really should be _downsample, but i dont wanna fix code right now
    data_test_upsample = pd.concat([downsampled_ham, data_test[data_test['y'] == 1]])

    if vectorizer is not None:
        data_train_features = vectorizer.fit_transform(data_train[text_field])
        data_train_y = data_train[label_name]
        data_test_features = vectorizer.transform(data_test[text_field])
        data_test_y = data_test[label_name]
        data_test_upsample_features = vectorizer.transform(data_test_upsample[text_field])
        data_test_upsample_y = data_test_upsample[label_name]
    else:
        # LOAD word to vec
        data_train_features = word2vec_transform(data_train, embeddings)
        data_train_y = data_train[label_name]
        data_test_features = word2vec_transform(data_test, embeddings)
        data_test_y = data_test[label_name]
        data_test_upsample_features = word2vec_transform(data_test_upsample, embeddings)
        data_test_upsample_y = data_test_upsample[label_name]
        
    model.fit(data_train_features, data_train_y)
    y_preds = model.predict_proba(data_test_features)[:, 1]
    fpr, tpr, threshold = metrics.roc_curve(data_test_y, y_preds)
    y_preds_upsample = model.predict_proba(data_test_upsample_features)[:, 1]
    fpr_upsample, tpr_upsample, threshold_upsample = metrics.roc_curve(data_test_upsample_y, y_preds_upsample)
    return metrics.auc(fpr, tpr), metrics.auc(fpr_upsample, tpr_upsample)

In [31]:
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 10), vectorizer=count_vect)

(0.9837854603125878, 0.9847946000870953)

In [32]:
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 10), vectorizer=tfidf_vect)

(0.9905641674980176, 0.9915263463492524)

In [33]:
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 10), vectorizer=bigram_count_vect)

(0.9389244877599571, 0.9391239657424881)

In [34]:
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 10), vectorizer=bigram_tfidf_vect)

(0.9413481953290871, 0.9391058208738567)

In [35]:
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 10), vectorizer=ngram_count_vect)

(0.9849045864988616, 0.984994193642038)

In [36]:
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 10), vectorizer=ngram_tfidf_vect)

(0.9906792776200344, 0.9927964871534329)

In [37]:
# defaults to word 2 vec
train_test_run(data_train, data_test, LogisticRegression(penalty='l2', C = 1), vectorizer = None)

(0.9267899623973601, 0.9250072579474525)